# 利用ELF文件分析Flash和Sram占用
输入文件：  
1.linker.cmd是链接依赖文件，决定不同section在FLASH还是SRAM中，或者兼而有之。  
2.iot_ap.elf是对应的ELF文件，可以获取不同section大小，以及symbol属性。

输出两种形式：  
1.页内显示top_counts个最大symbol  
2.将每个section的symbol输出到一个csv文件中  

In [ ]:
import os
import re
import pandas as pd
import numpy as np

## 1 预处理：分析linker.cmd，获取Section，获取Symbol

通过分析linker.cmd，获取section是保存在FLASH， SRAM还是都保存。

In [ ]:
linker_cmd = 'linker.cmd'
elf_file = 'iot_ap.elf'

elf_data = {'flash_capacity':274*1024, 'flash_size':0, 'sram_capacity':71*1024, 'sram_size':0}

linker_lines = open(linker_cmd, 'r').readlines()
linker_cmd_fmt = ' *(?P<section>.*).*:.*'
#sections = {'name':[], 'flash':[], 'sram':[]}
section_location_map = {}
section_name = ''
section_flash = 0
section_sram = 0

for line in linker_lines:

    if '> FLASH' in line:
        section_flash = 1
    if '> SRAM' in line:
        section_sram = 1
    m = re.match(linker_cmd_fmt, line)
    if not m:
        continue

    if section_name != 'null' and section_name != '':
        section_location_map[section_name] = [section_flash, section_sram]

    section_flash = 0
    section_sram = 0
    section_name = m.group('section')
    section_name = section_name.split(' ')[0]

#for i in section_location_map:
#    print i, section_location_map[i]

通过分析elf的Section Headers得知：section_location_map的section名称和elf_lists中section index的对应关系。

In [ ]:
tmp = os.popen('readelf -S %s' % elf_file).readlines()
section_fmt = ' *\[(?P<index>.*)\] (?P<message>.*)'
flash_size_total = 0
sram_size_total = 0
section_data = {}
for line in tmp:
    m = re.match(section_fmt, line)
    if not m:
        continue
    section_index = m.group('index').strip()
    message = m.group('message')
    section_line_list = re.split(r'\s+', message)
    
#  [Nr] Name              Type            Addr     Off    Size   ES Flg Lk Inf Al
#  [ 1] text              PROGBITS        0ffaf800 0000b8 024f64 00 WAX  0   0  4

    if section_line_list[4] != 'Size':
        section_line_list[4] = int(section_line_list[4], 16)

        section_name = section_line_list[0]
        section_addr = section_line_list[2]
        section_size = section_line_list[4]

        if section_location_map.has_key(section_name):
            section_data[section_index] = {'size':section_size, 
                                              'addr':section_addr, 
                                              'name':section_name, 
                                              'flash':section_location_map[section_name][0],
                                              'sram':section_location_map[section_name][1]}
#for i in section_data:
#    print section_data[i]


通过分析elf文件，获取所有符号的相关信息：size, address, section, name。

In [ ]:
elf_summary = elf_file.split('.')[0]
symbol_lists = []
top_counts = 20
data_sections = {}

if not os.path.exists(elf_summary):
    os.mkdir(elf_summary)

#elf_summary_object = open(elf_summary, 'wb')

tmp = os.popen('readelf -s %s' % elf_file).readlines()
elf_symbol_fmt = ' *(?P<num>[0-9]*): (?P<value>[0-9abcdef]*) *(?P<size>[0-9]*).*'
for line in tmp:
    m = re.match(elf_symbol_fmt, line)
    if not m:
        continue
    #num = m.group('num')
    symbol_line_list = re.split(r'\s+', line)

    if symbol_line_list[3][0:2] == '0x':
        symbol_line_list[3] = int(symbol_line_list[3][2:], 16)
    #       size             address            section             name
    symbol_name = symbol_line_list[8]
    symbol_size = symbol_line_list[3]
    symbol_section = symbol_line_list[7]
    symbol_addr = symbol_line_list[2]
    symbol_lists.append([symbol_size, symbol_addr, symbol_section, symbol_name])
#elf_summary_object.writelines(tmp)
#elf_summary_object.close()

## 2 处理数据：按Section分类，对每Section数据降序排列
1.获取每个Symbol的size， address， section， name。  
2.计算每个section实际占用空间：Flash或者Sram。  

In [ ]:
elf_data['flash_size'] = 0
elf_data['sram_size'] = 0

symbol_data = pd.DataFrame(np.asarray(symbol_lists), columns=['size', 'address', 'section', 'name'])
symbol_data['size'] = symbol_data['size'].astype(int)
symbol_data.sort_values(by=['size'], ascending=False).to_csv('%s/%s.csv' % (elf_summary, 'top_all'))

for section in section_data:
    section_data[section]['symbol_data'] = symbol_data[symbol_data['section'] == section].sort_values(by=['size'], ascending=False)

for section in section_data:
    section_size = 0
    #print section_data[section]['name'], section_data[section]['symbol_data']['size'].astype(int).sum(), section_data[section]['size']
    section_data[section]['size'] = section_data[section]['symbol_data']['size'].astype(int).sum()

for section in section_data:
    if section_data[section]['flash']:
        elf_data['flash_size'] += section_data[section]['size']
    if section_data[section]['sram']:
        elf_data['sram_size'] += section_data[section]['size']

## 3 输出结果
1.占用的总Flash或者Sram空间。  
2.将每个Section的symbol按大小降序排列输出到csv文件。  

In [ ]:
print 'Total FLASH usage: %d / %d ' % (elf_data['flash_size'], elf_data['flash_capacity'])
print 'Total SRAM usage: %d / %d' % (elf_data['sram_size'], elf_data['sram_capacity'])

In [ ]:
for section in section_data:
    print '\n', section_data[section]['name'], section_data[section]['size'], '(', 'FLASH,' if section_data[section]['flash'] else '', 'SRAM' if section_data[section]['sram'] else '', ')'
    print section_data[section]['symbol_data'].head(top_counts)
    section_data[section]['symbol_data'].to_csv('%s/%s.csv' % (elf_summary, section_data[section]['name']))